# 07 Morris Counting

Here is an implementation of Morris counting by hand, and then with machine learning. For this notebook, the random bits are preprocessed to be in the correct scale so that a random bit of 0 means to increment the counter and anything else means not to. This means that essentially, this network just needs to learn the `and` function and increment the counter whenever the string has a 1 and the random bit is 0.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/My\ Drive/CS281\ Final\ Project

## Package Loading

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from random import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
device = 'cuda'

import pickle

## Manual Counter Implementation

In [0]:
def morris_count(seq):
    '''Takes a sequence, and returns the log output of a Morris counter as well as the random bits used to make the counter'''
    x = 0 
    bits = []
    for i in seq:
        noise = np.random.randint(0, 2**x)
        bits.append(noise)
        if i == 1 and noise == 0:
            x += 1
    return x, bits

## Model Definition

In [0]:
class Counter():
    def __init__(self, hidden, input_size=1):
        '''hidden is the number of hidden variables to use per cell'''
        self.hidden = hidden
        #this LSTM goes from input [batch x length x 1] to output [batch x length x hidden]
        self.lstm = nn.LSTM(hidden_size=hidden, batch_first=True, input_size=input_size).double().cuda() 

        #this matrix transforms from [1 x hidden] to [1 x 1]
        self.combine = torch.randn([hidden,1], dtype=float, device=device, requires_grad=True) 

        params = list(self.lstm.parameters())
        params.append(self.combine)
        self.optimizer = optim.Adam(params)

    @staticmethod
    def convert_sequence(seq, input_size=1):
        '''converts a set of sequences with the same length from array or numpy into a correctly formatted tensor.'''
        seq = torch.tensor(seq, device=device).double()
        seq = seq.reshape([len(seq), -1, input_size])
        return seq

    def predict(self, sequence, target):
        '''takes a tensor, predicts the sum of the tensor, and compares to the target sum of the tensor. 
        Returns the loss and the predicted sum'''

        h = torch.zeros([1,sequence.shape[0],self.hidden]).double().cuda()
        c = torch.zeros([1,sequence.shape[0],self.hidden]).double().cuda()

        o, _ = self.lstm(sequence)
        add = torch.sigmoid(o @ self.combine)
        count = add.sum(1).squeeze(1)
        loss = (count - target).pow(2)
        return loss, count

    def predict_multilength(self, sequences, target):
        '''Takes a list of batches of tensors of different length. Predicts on each batch. Sums up the loss. Reduces to a single mean'''
        loss = torch.tensor(0, device=device).double()
        count= torch.tensor(0, device=device).double()
        for s,t in zip(sequences, target):
            res    = self.predict(s,t)[0]
            count += res.shape[0]
            loss  += res.sum()
        return loss / count

    @staticmethod
    def true_sum(sequence):
        '''Determines the true sums of a batch of sequences to train against'''
        comp = (sequence >= 0).float()
        comp = comp.sum(1).squeeze(1)
        return comp

## Data Generation

In [0]:
def add_bits(dataset):
    '''takes a dataset of sequences and runs the morris counter to get the random bits and the true labels'''
    X_set = []
    Y_set = []
    for chunk in dataset:
        converted = []
        target = []
        for s in chunk:
            x, bits = morris_count(s)
            converted.append(np.array([s,bits]).T)
            target.append(x)
        converted = np.array(converted).astype(float)
        converted = Counter.convert_sequence(converted, input_size=2)
        target = torch.tensor(target, device=device).double()
        X_set.append(converted)
        Y_set.append(target)
    return X_set, Y_set


In [0]:
def generate_data(length, total):
    counts = np.random.dirichlet((np.arange(length)+1)**2) * total * 0.9
    counts = np.round(counts).astype(int)

    train_set = []
    val_set = []
    test_set = []

    for i in range(1,length+1):
        if counts[i-1] == 0:
            continue
        seqs = np.random.randint(0,2, size=[counts[i-1],i])
        seqs = np.unique(seqs, axis=0)
        try:
            train, val = train_test_split(seqs, test_size=2/9, shuffle=True)
            train_set.append(train)
            val_set.append(val)
        except ValueError:
            continue

    counts = np.random.dirichlet((np.arange(length, 2*length)+1)**2) * total * 0.1
    counts = np.round(counts).astype(int)
    for i in range(length):
        if counts[i] == 0:
            continue
        seqs = np.random.randint(0,2, size=[counts[i],i+length+1])
        seqs = np.unique(seqs, axis=0)
        test_set.append(seqs)

    train_set = add_bits(train_set)
    val_set = add_bits(val_set)
    test_set = add_bits(test_set)

    return train_set, val_set, test_set


In [0]:
#generate all the strings and partition into train and test
length = 64
hidden = 10
depth = 100000

output_folder = "Part-7-Outputs"

In [0]:
# train, val, test = generate_data(length,depth)

# with open("%s/Data.pickle"%output_folder, "wb") as f:
#     pickle.dump([train, val, test], f)

In [0]:
with open("%s/Data.pickle"%output_folder, "rb") as f:
    train, val, test = pickle.load(f)

In [0]:
train, train_y = train
val, val_y = val
test, test_y = test

In [0]:
trainsize = sum([x.shape[0] for x in train])
valsize = sum([x.shape[0] for x in val])
testsize = sum([x.shape[0] for x in test])
print(trainsize, valsize, testsize)

total = trainsize+valsize+testsize
print("Total:",total) 
print("Fraction %.3f %.3f %.3f"%(trainsize/total, valsize/total, testsize/total))

print("Train    string range: %d-%d"%(min([x.shape[1] for x in train]), max([x.shape[1] for x in train])))
print("Validate string range: %d-%d"%(min([x.shape[1] for x in val]), max([x.shape[1] for x in val])))
print("Test     string range: %d-%d"%(min([x.shape[1] for x in test]), max([x.shape[1] for x in test])))

69921 20011 9995
Total: 99927
Fraction 0.700 0.200 0.100
Train    string range: 1-64
Validate string range: 1-64
Test     string range: 65-128


## Model Training

In [0]:
#train over all the training data
model = Counter(hidden=hidden, input_size=2)

history = []
best = float('inf')
patience = 10
tol = 0.001
count = 0

for epoch in range(1000000): 
    # shuffle(train)
    # shuffle(val)
    if epoch % 100 == 0:
        train_loss = model.predict_multilength(train, train_y).item()
        with torch.no_grad():
            val_loss = model.predict_multilength(val, val_y).item()
        history.append([train_loss, val_loss])
        print("Epoch: %5d. Train Loss: %7.3f. Validation Loss: %7.3f"%(epoch, train_loss, val_loss))

        if val_loss + tol < best:
            best = val_loss
            count = 0
        else:
            count += 1
        if count >= patience:
            break

    #take the average loss over all the train data
    loss = model.predict_multilength(train, train_y)   
    #and update
    model.optimizer.zero_grad()
    loss.backward(retain_graph=True)
    model.optimizer.step()

# history = np.array(history)
# np.save("%s/Train-History"%output_folder, history)
# torch.save(model, "%s/Model"%output_folder)

# #display testing results
# loss = model.predict_multilength(test)
# print("Average Test Loss:", loss.item())

## Results Evaluation

In [0]:
history = np.load("%s/Train-History.npy"%output_folder)
train_loss = history[:,0]
val_loss = history[:,1]
model = torch.load("%s/Model"%output_folder)

In [0]:
loss = model.predict_multilength(test, test_y)
print("Average Test Loss:", loss.item())

Average Test Loss: 0.0014881722468963487


In [0]:
#sample sequence and random bits
seq = test[-1][0].cpu().numpy()
print(seq[:,0])
print(seq[:,1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0.
 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 0. 1. 0. 1. 0. 1.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  1.  0.  0.
  3. 10.  6.  7.  7. 14. 10.  9.  8.  5.  7. 10. 12. 10.  2.  0. 30. 26.
 24. 14. 26.  8.  9.  9. 21.  2.  7. 22. 17. 21. 30. 11. 18. 16. 14. 25.
  7. 28. 21. 14. 28. 29.  3. 21. 16. 16.  2. 17. 19. 19.  1. 30. 10. 19.
  0. 59. 53. 19. 60.  0. 60.  2. 26. 22. 57. 59.  5. 63.  8. 43. 55. 28.
 38. 32. 20. 33.  8. 20.  6.  6. 21. 43. 19. 58.  0. 18. 43. 22.  9. 19.
 13.  9. 18. 30. 60. 61. 29. 61.  6. 28. 40. 14. 59. 10. 17. 51. 47.  2.
 22. 30.]


In [0]:
#print the true sum and predicted sum for one string of each length
for chunk, chunk_y in zip(test, test_y):
    ind = np.random.choice(chunk.shape[0], 1)
    sample = chunk[ind]
    true_sum = chunk_y[ind]
    print("Length: %3d Sum: %3d Pred: %7.4f"%(sample.shape[1], true_sum, model.predict(sample, true_sum)[1].item()))

Length:  65 Sum:   5 Pred:  4.9960
Length:  66 Sum:   4 Pred:  4.0147
Length:  67 Sum:   5 Pred:  4.9950
Length:  68 Sum:   4 Pred:  3.9830
Length:  69 Sum:   5 Pred:  5.0260
Length:  70 Sum:   4 Pred:  4.0249
Length:  71 Sum:   4 Pred:  4.0468
Length:  72 Sum:   5 Pred:  5.0083
Length:  73 Sum:   4 Pred:  3.9956
Length:  74 Sum:   7 Pred:  7.0337
Length:  75 Sum:   5 Pred:  4.9964
Length:  76 Sum:   6 Pred:  6.0039
Length:  77 Sum:   6 Pred:  5.9947
Length:  78 Sum:   5 Pred:  5.0316
Length:  79 Sum:   5 Pred:  5.0141
Length:  80 Sum:   4 Pred:  4.0696
Length:  81 Sum:   5 Pred:  5.0028
Length:  82 Sum:   5 Pred:  4.9955
Length:  83 Sum:   5 Pred:  5.0115
Length:  84 Sum:   6 Pred:  5.9895
Length:  85 Sum:   6 Pred:  6.0095
Length:  86 Sum:   4 Pred:  4.0456
Length:  87 Sum:   4 Pred:  4.0460
Length:  88 Sum:   4 Pred:  4.0271
Length:  89 Sum:   6 Pred:  6.0338
Length:  90 Sum:   4 Pred:  4.0300
Length:  91 Sum:   5 Pred:  5.0131
Length:  92 Sum:   4 Pred:  4.0445
Length:  93 Sum:   6